In [1]:
%cd ../..
%reload_ext autoreload
%autoreload 2

/mnt/d/ownCloud/Uni/Semester Ma 5/Advanced Deep Learning for Robotics (IN2349)/Project/tum-adlr-ws20-9


In [2]:
import pyspiel
import numpy as np

from alpha_one.model.model_manager import AlphaOneCheckpointManager, OpenSpielCheckpointManager, CFRCheckpointManager
from alpha_one.utils.play import VerboseGameMachine
from alpha_one.utils.mcts import MCTSConfig
from alpha_one.utils.mcts_II import IIGMCTSConfig
from alpha_one.utils.state_to_value import state_to_value
from alpha_one.utils.statemask import get_state_mask
from alpha_one.model.agent import DirectInferenceAgent, IIGMCTSAgent, OmniscientAgent, DMCTSAgent, CFRAgent
from alpha_one.game.observer import OmniscientObserver

Instructions for updating:
non-resource variables are not supported in the long term


# 1. Game Setup

In [3]:
game_name = 'leduc_poker'
run_name = 'LP-local-46'

game = pyspiel.load_game(game_name)

In [11]:
game_machine = VerboseGameMachine(game)
game_machine.new_game()
game_machine.play_action(0)
game_machine.play_action(1)
game_machine.play_action(2)
game_machine.play_action(2)
game_machine.play_action(1)
game_machine.get_observations()
game_machine.play_action(5)
game_machine.play_action(2)
game_machine.play_action(2)
game_machine.play_action(1)
game_machine.get_observations()
game_machine.list_player_actions()

Omniscient Observation: [Round 2][Player: -1][Pot: 10][Money: 95 95[Private: 01][Ante: 5 5]
Omniscient Observation: [Round 2][Player: 0][Pot: 0][Money: 100 100[Private: 01][Ante: 13 13][Public: 5]
Current Player: -4
Legal actions: 


[]

In [603]:
model_manager = AlphaOneCheckpointManager(game_name, run_name)
observation_model, game_model = model_manager.load_checkpoint(-1)

INFO:tensorflow:Restoring parameters from /home/tobias/Uni/Semester Ma 5/Advanced Deep Learning for Robotics (IN2349)/Project/model_saves/leduc_poker/LP-local-46-observation_model/checkpoint-18
INFO:tensorflow:Restoring parameters from /home/tobias/Uni/Semester Ma 5/Advanced Deep Learning for Robotics (IN2349)/Project/model_saves/leduc_poker/LP-local-46-game_model/checkpoint-18


# 2. Model Setup

## 2.1. AlphaOne

In [604]:
n_previous_observations = 3

In [682]:
alpha_one_mcts_config = IIGMCTSConfig(
    uct_c=10000,
    max_mcts_simulations=100,
    temperature=1,
    optimism=0.001,
    n_previous_observations=3,
    use_reward_policy=True,
    alpha_one=True,
    state_to_value=state_to_value(game_name))

In [683]:
direct_inference_agent = DirectInferenceAgent(observation_model, n_previous_observations=n_previous_observations)
alpha_one_agent = IIGMCTSAgent.from_config(game, observation_model, game_model, alpha_one_mcts_config)

## 2.2. D-MCTS

In [259]:
run_name_d_mcts = "LP-DMCTS"

In [260]:
model_manager_dmcts = OpenSpielCheckpointManager(game_name, run_name_d_mcts)

dmcts_model = model_manager_dmcts.load_checkpoint(-1)
dmcts_model_untrained = model_manager_dmcts.load_checkpoint(0)

INFO:tensorflow:Restoring parameters from /home/tobias/Uni/Semester Ma 5/Advanced Deep Learning for Robotics (IN2349)/Project/model_saves/leduc_poker/LP-DMCTS/checkpoint-12
INFO:tensorflow:Restoring parameters from /home/tobias/Uni/Semester Ma 5/Advanced Deep Learning for Robotics (IN2349)/Project/model_saves/leduc_poker/LP-DMCTS/checkpoint-0


In [531]:
dmcts_mcts_config = MCTSConfig(uct_c=1000, 
                               max_mcts_simulations=100,
                               temperature=1,
                               determinized_MCTS=True, 
                               omniscient_observer=True,
                               use_reward_policy=True)

d_mcts_agent = DMCTSAgent(dmcts_model, dmcts_mcts_config)
d_mcts_agent_untrained = DMCTSAgent(dmcts_model_untrained, dmcts_mcts_config)
d_mcts_random_rollout_agent = DMCTSAgent(None, dmcts_mcts_config, n_rollouts=1)

## 2.3. Omniscient

In [242]:
run_name_omniscient = "LP-12"

In [245]:
model_manager_omniscient = OpenSpielCheckpointManager(game_name, run_name_omniscient)

omniscient_model = model_manager_omniscient.load_checkpoint(-1)

INFO:tensorflow:Restoring parameters from /home/tobias/Uni/Semester Ma 5/Advanced Deep Learning for Robotics (IN2349)/Project/model_saves/leduc_poker/LP-12/checkpoint-6


In [256]:
omniscient_mcts_config = MCTSConfig(5, 100, 1, use_reward_policy=True, omniscient_observer=True)

omniscient_agent_untrained = OmniscientAgent(game, omniscient_mcts_config)
omniscient_agent = OmniscientAgent(game, omniscient_mcts_config, model=omniscient_model)

## 2.4. Blind Model

In [515]:
run_name_blind = "LP-12-blind-1"

In [516]:
model_manager_blind = OpenSpielCheckpointManager(game_name, run_name_blind)

blind_model = model_manager_blind.load_checkpoint(-1)

INFO:tensorflow:Restoring parameters from /home/tobias/Uni/Semester Ma 5/Advanced Deep Learning for Robotics (IN2349)/Project/model_saves/leduc_poker/LP-12-blind-1/checkpoint-999


In [517]:
blind_agent = DirectInferenceAgent(blind_model)

## 2.5. CFR

In [521]:
run_name_cfr = 'LP-CFR-2'

In [528]:
model_manager = CFRCheckpointManager(game_name, run_name_cfr)
cfr_policy_table = model_manager.load_checkpoint(-1)
cfr_agent = CFRAgent(cfr_policy_table, temperature=1)

# 3. Investigate specific Game Scenarios

In [607]:
omniscient_observer = OmniscientObserver(game)

In [608]:
game_machine = VerboseGameMachine(game)

In [770]:
game_machine.new_game()

game_machine.play_action(5)
game_machine.play_action(0)
game_machine.play_action(2)
game_machine.play_action(1)
#game_machine.play_action(1)
game_machine.play_action(4)
game_machine.play_action(2)
#game_machine.play_action(2)
#game_machine.play_action(1)

In [771]:
game_machine.list_player_actions()

Current Player: 1
Legal actions: 
  - 0: Fold
  - 1: Call
  - 2: Raise


[0, 1, 2]

## 3.1. Direct Inference

In [772]:
obs = [game_machine.information_set_generator.get_padded_observation_history(n_previous_observations)]

information_set = game_machine.information_set_generator.calculate_information_set()
state_mask, _ = get_state_mask(alpha_one_mcts_config.state_to_value, information_set)

In [773]:
game_model.inference([omniscient_observer.get_observation_tensor(information_set[0])], [game_machine.get_state().legal_actions_mask()])

[array([[0.11920792]], dtype=float32),
 array([[0.0964878 , 0.38241032, 0.52110183]], dtype=float32)]

In [774]:
value, policy = observation_model.inference(obs,  [state_mask])

In [775]:
policy[0][state_mask]

array([0.33953798, 0.06093471, 0.3116335 , 0.28789374], dtype=float32)

In [776]:
value

array([[0.50686806]], dtype=float32)

## 3.2. AlphaOne

In [798]:
alpha_one_agent.next_move(game_machine.information_set_generator)

(1, array([0.33509969, 0.33509969, 0.32980062]))

In [797]:
alpha_one_agent.get_last_state_policy()

array([3.25777950e-03, 2.95529772e-04, 6.03452244e-04, 9.95843238e-01])

## 3.3. Omniscient

In [792]:
omniscient_agent.next_move(game_machine.get_state())

(0, array([0.97810967, 0.0179147 , 0.00397562]))

## 3.4. D-MCTS 

In [793]:
d_mcts_agent.next_move(game_machine.get_information_set_generator())

(0, array([0.89957579, 0.09481459, 0.00560962]))

## 3.5. Blind

In [794]:
blind_agent.next_move(game_machine.get_state())

(0, array([0.8333642 , 0.08430433, 0.08233147], dtype=float32))

## 3.6. CFR

In [795]:
cfr_agent.next_move(game_machine.get_state())

(0, array([0.83984927, 0.00926617, 0.15088456]))